In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

train_dir = "/content/drive/My Drive/Colab Notebooks/dogs"
num_classes = 70
image_size = (299, 299)
batch_size = 32
initial_learning_rate = 0.001
fine_tune_learning_rate = 1e-5
epochs = 30

training_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = training_data.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset='training'
)

valid_generator = training_data.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation'
)
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(image_size[0], image_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss=loss_fn, metrics=["accuracy"])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=10,
    callbacks=[early_stop]
)

for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), loss=loss_fn, metrics=["accuracy"])

history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs - 10,
    callbacks=[early_stop]
)

model.save("dogclassification_improved.h5")


Found 6798 images belonging to 70 classes.
Found 1658 images belonging to 70 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 3274s 15s/step - accuracy: 0.7183 - loss: 1.8482 - val_accuracy: 0.8689 - val_loss: 1.2324
Epoch 2/10
  1/212 ━━━━━━━━━━━━━━━━━━━━ 34:19 10s/step - accuracy: 0.8438 - loss: 1.2053

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


212/212 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.8438 - loss: 1.2053 - val_accuracy: 0.8787 - val_loss: 1.2110
Epoch 3/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 2677s 12s/step - accuracy: 0.9046 - loss: 1.0684 - val_accuracy: 0.8658 - val_loss: 1.1869
Epoch 4/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - accuracy: 0.8750 - loss: 1.0843 - val_accuracy: 0.8640 - val_loss: 1.1840
Epoch 5/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 2642s 12s/step - accuracy: 0.9186 - loss: 1.0263 - val_accuracy: 0.8860 - val_loss: 1.1500
Epoch 6/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 512s 2s/step - accuracy: 0.8438 - loss: 1.1582 - val_accuracy: 0.8762 - val_loss: 1.1349
Epoch 7/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 2663s 12s/step - accuracy: 0.9312 - loss: 0.9970 - val_accuracy: 0.8652 - val_loss: 1.2110
Epoch 8/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 510s 2s/step - accuracy: 0.9375 - loss: 1.0084 - val_accuracy: 0.8572 - val_loss: 1.2289
Epoch 9/10
212/212 ━━━━━━━━━━━━━━━━━━━━ 2544s 12s/step - accuracy: 0.9286 - loss: 1.0015 - val_accurac

# New section